In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [2]:
import os
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

/home/dulanj/miniconda3/envs/mlops9/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/dulanj/miniconda3/envs/mlops9/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [ ]:
year = 2021
month = 2
input_file = f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet'
output_file_name = f"predictions_{year:04d}-{month:02d}_fvh_data.parquet"
output_file = os.path.join("output", output_file_name)
df = read_data(input_file)

In [ ]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [ ]:
import numpy as np
mean  = round(np.mean(y_pred), 2)
print(f"Answer for Q1: {mean}")

In [ ]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [ ]:
data = {
    'ride_id': df['ride_id'].values,
    'predictions': y_pred
}
df_result = pd.DataFrame(data)

In [ ]:
import os
if not os.path.exists('output'):
    os.makedirs('output')
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [ ]:
# print size of the file
size_in_bytes = os.path.getsize(output_file)
print(f"Answer for Q2 : File size in MB: {size_in_bytes/1000000}")